In [1]:
#importar librerias
import pandas as pd
pd.set_option('display.max_columns', None)

import re

from tqdm import tqdm

import requests
import time
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [11]:
#abrimos el navegador de Selenium
navegador = webdriver.Chrome()

In [12]:
#maximizamos el navegador
navegador.maximize_window()

In [13]:
#vamos al url
navegador.get('https://nomadlist.com/userApi.php?action=login_by_email&hash=91e7c90f5dcbfc27710077645e4f4841b8949523')

In [14]:
#abrimos el menu
menu = navegador.find_element_by_xpath('/html/body/div[3]/div/div[1]/span')
menu.click()

In [15]:
#buscamos el home
home = navegador.find_element_by_xpath('/html/body/div[5]/a[1]')
home.click()

In [16]:
#scroll al final de la página
n_ciudades = len(navegador.find_elements_by_tag_name('li'))

while n_ciudades<700:
    navegador.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(4)
    n_ciudades = len(navegador.find_elements_by_tag_name('li'))

In [8]:
#lista de ciudades
ciudades = []
lst_ciudades = []

In [10]:
#Si solo es un archivo de checkpoint CSV
ciudades_df = pd.read_csv('nomadlist_cities.csv').drop('Unnamed: 0', axis=1)

#generamos una lista de las ciudades para ir haciendo el append
ciudades = ciudades_df.to_dict('records')

#total de ciudades ya escrapeadas
print(len(ciudades))

#generamos una lista para verificar si ya está añadida la ciudad
lst_ciudades = [i['City']+'-'+i['Country'] for i in ciudades]

645


In [20]:
%%time

saver = 0

for i in tqdm(range(1,550)):
    try:
        #seleccionamos una ciudad
        cada_ciudad = navegador.find_element_by_xpath(f'/html/body/div[7]/ul/li[{i}]')

        #diccionario de la ciudad
        data_ciudad = {}

        #nombre de la ciudad
        data_ciudad['City'] = cada_ciudad.find_element_by_class_name('itemName').text
        try:
            data_ciudad['Country'] = cada_ciudad.find_element_by_class_name('itemSub').text
        except:
            data_ciudad['Country'] = data_ciudad['City']

        if data_ciudad['City']+'-'+data_ciudad['Country'] in lst_ciudades:
            #print(data_ciudad['City']+'-'+data_ciudad['Country']+'---Ya está en lista')
            pass
        else:
            cada_ciudad.click()
            time.sleep(1)

            data_ciudad['Photo'] = navegador.find_element_by_xpath('/html/body/div[32]/div[3]/div[1]/img').get_attribute('src')

            lst_ciudades.append(data_ciudad['City']+'-'+data_ciudad['Country'])
            #sacamos información de la ciudad

            #tab de scores
            nomad_guide = navegador.find_element_by_xpath('/html/body/div[32]/div[3]/div[2]/div[4]/ul/h3[1]')
            nomad_guide.click()

            time.sleep(1)

            tabla = navegador.find_element_by_xpath('//*[@id="body"]/div[32]/div[3]/div[2]/div[5]/div/div[1]/table/tbody')
            data = tabla.find_elements_by_tag_name('tr')

            for i in data:
                try:
                    titulo = i.find_element_by_class_name('key').text
                    titulo = re.sub('[^a-zA-Z ]',"",titulo).strip()
                except:
                    pass

                lst = ['Overall Score', 'Quality of life score', 'Family score', 'Cost',
                       'Fun', 'Safety', 'Education level', 'English speaking', 'People density /km²', 
                       'Walkability', 'Peace', 'Traffic safety', 'Hospitals', 'Happiness', 'Nightlife', 
                       'Free WiFi in city', 'Places to work from', 'A/C or heating', 
                       'Friendly to foreigners', 'Freedom of speech', 'Racial tolerance', 
                       'Female friendly', 'LGBTQ friendly', 'Startup Score']
                if titulo in lst:
                    contenido = i.find_element_by_class_name('value').text
                    if titulo == 'Overall Score':
                        contenido = contenido.split('/')[0]
                    elif titulo == 'Cost':
                        contenido = contenido.split(' ')[2]
                        contenido = re.sub('[^0-9. ]',"",contenido)
                        titulo = 'Cost/month (USD)'
                    elif i.get_attribute('data-value') != None:
                        contenido = i.get_attribute('data-value')

                    data_ciudad[titulo] = contenido

            #tab de Nomad
            nomad_guide = navegador.find_element_by_xpath('/html/body/div[32]/div[3]/div[2]/div[4]/ul/h3[2]')
            nomad_guide.click()

            time.sleep(1)

            tabla = navegador.find_element_by_xpath('//*[@id="body"]/div[32]/div[3]/div[2]/div[5]/div/div[2]/table/tbody')
            data = tabla.find_elements_by_tag_name('tr')

            for i in data:
                try:
                    titulo = i.find_element_by_class_name('key').text
                    titulo = re.sub('[^a-zA-Z ]',"",titulo).strip()
                except:
                    pass

                lst = ['Best taxi app in country','Internet speed avg','Best wireless carrier','Best wireless carrier','Tipping','Cashless society','Safe tap water','Population']
                if titulo in lst:
                    contenido = i.find_element_by_class_name('value').text
                    if titulo == 'Internet speed avg':
                        contenido = contenido.split(' ')
                        tit = contenido[1].strip()
                        titulo = f'Internet {tit}'
                        contenido = contenido[0].strip()
                    else:
                        contenido = re.sub('[^a-zA-Z0-9 ]',"",contenido).strip().split(' ')[0]
                    data_ciudad[titulo] = contenido

            #tab de Weather
            nomad_guide = navegador.find_element_by_xpath('/html/body/div[32]/div[3]/div[2]/div[4]/ul/h3[7]')
            nomad_guide.click()

            time.sleep(1)

            tabla = navegador.find_element_by_xpath('/html/body/div[32]/div[3]/div[2]/div[5]/div/div[7]/div/table/tbody')
            data = tabla.find_elements_by_tag_name('tr')

            meses = data[0].text.split(' ')
            temp = [i.split('°')[0] for i in data[2].text.split(' ')[1:] if '°' in i]

            for i,e in enumerate(meses):
                data_ciudad[f'Temperature (°C) {e}'] = temp[i]

            #agregar la información a la lista principal de ciudades
            ciudades.append(data_ciudad)

            #cerrar ciudad
            close = navegador.find_element_by_xpath('//*[@id="body"]/div[33]/div')
            close.click()
            
            saver +=1

            time.sleep(1)

            if saver % 10==0:
                ciudades_df = pd.DataFrame(ciudades)
                ciudades_df.to_csv('nomadlist_cities2.csv')
    except:
        pass

100%|██████████| 549/549 [2:40:35<00:00, 17.55s/it]  

CPU times: user 59.7 s, sys: 4.02 s, total: 1min 3s
Wall time: 2h 40min 35s


In [21]:
ciudades_df = pd.DataFrame(ciudades)
ciudades_df

,City,Country,Photo,Overall Score,Quality of life score,Family score,Cost/month (USD),Internet Mbps,Fun,Safety,Education level,English speaking,Walkability,Peace,Traffic safety,Hospitals,Happiness,Nightlife,Free WiFi in city,Places to work from,Friendly to foreigners,Freedom of speech,Racial tolerance,Female friendly,LGBTQ friendly,Startup Score,Best taxi app in country,Best wireless carrier,Tipping,Cashless society,Safe tap water,Population,Temperature (°C) Jan,Temperature (°C) Feb,Temperature (°C) Mar,Temperature (°C) Apr,Temperature (°C) May,Temperature (°C) Jun,Temperature (°C) Jul,Temperature (°C) Aug,Temperature (°C) Sep,Temperature (°C) Oct,Temperature (°C) Nov,Temperature (°C) Dec
0,Lisbon,Portugal,https://nomadlist.com/assets/img/places/lisbon...,4.68,4,4,2006,27,4,Great,3,4,5,4,3,2,3,3,3,5,4,5,3,4,5,3,Uber,Vodafone,No,Yes,Yes,550000,14,16,19,18,22,21,23,25,24,21,17,16
1,"Canggu, Bali",Indonesia,https://nomadlist.com/assets/img/places/canggu...,4.61,4,4,1247,19,4,Great,3,4,3,3,2,NaN,3,4,4,5,5,3,4,4,4,3,GoJEK,XL,No,No,No,30000,30,30,30,30,30,28,27,27,27,29,30,31
2,Mexico City,Mexico,https://nomadlist.com/assets/img/places/mexico...,4.46,4,3,1278,19,4,Okay,3,3,5,2,3,NaN,4,4,3,5,4,3,2,3,5,3,Uber,Telcel,No,No,No,8900000,19,22,24,24,25,22,21,22,21,21,21,20
3,Chiang Mai,Thailand,https://nomadlist.com/assets/img/places/chiang...,4.45,4,3,1056,24,4,Great,3,3,5,3,2,5,4,3,4,5,4,3,2,4,3,3,Grab,AIS,No,Yes,No,400000,29,32,36,37,35,32,31,31,31,31,30,28
4,Belgrade,Serbia,https://nomadlist.com/assets/img/places/belgra...,4.45,4,4,1423,29,4,Good,3,4,5,3,3,NaN,3,5,4,5,4,4,3,4,1,3,CarGo,Telenor,No,Yes,Yes,1400000,1,8,15,18,20,28,28,32,26,23,16,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
966,La Serena,Chile,https://nomadlist.com/assets/img/places/la-ser...,3.21,3,3,1884,12,NaN,Okay,4,2,5,4,3,NaN,4,3,2,5,3,4,3,2,2,2,Uber,Movistar,No,Yes,No,210000,22,23,21,20,18,17,16,17,15,17,19,20
967,Managua,Nicaragua,https://nomadlist.com/assets/img/places/managu...,3.25,3,2,1179,7,2,Bad,2,2,3,3,2,3,3,2,1,5,4,4,NaN,3,3,2,NaN,ATT,No,NaN,No,1000000,31,31,32,33,32,32,31,32,31,30,31,31
968,Fukuoka,Japan,https://nomadlist.com/assets/img/places/fukuok...,3.22,4,3,2961,42,5,Great,4,1,5,4,3,4,3,3,3,5,2,4,3,4,3,2,Line,Soft,No,Yes,Yes,1500000,12,13,14,19,24,26,28,31,29,23,19,14
969,"Riverside, CA",United States,https://nomadlist.com/assets/img/places/rivers...,3.22,4,3,3360,38,4,Good,4,5,1,3,3,4,4,3,4,1,3,4,3,4,5,3,Uber,Verizon,No,Yes,Yes,320000,17,9,22,13,23,15,32,32,16,29,12,21


In [22]:
ciudades_df.to_csv('nomadlist_cities2.csv')